In [ ]:
import os
import shutil

# Definindo os diretórios
source_dir = 'D:/CloudSEN12+/p509/high/train/Todas'  # Caminho da pasta 'ROI'
destination_dir = 'D:/CloudSEN12+/p509/high/train/Todas'  # Caminho do diretório de destino

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Percorre todas as subpastas dentro da pasta 'ROI'
for root, dirs, files in os.walk(source_dir):
    for file in files:
        # Verifica se o arquivo é uma imagem (pode adicionar mais extensões conforme necessário)
        if file.endswith(('.tif')):
            # Caminho completo do arquivo
            file_path = os.path.join(root, file)
            
            # Copia o arquivo para o diretório de destino
            shutil.copy(file_path, destination_dir)

print("Imagens copiadas com sucesso!")


In [7]:
import os
import shutil

# Definindo os diretórios
source_dir = 'D:/CloudSEN12+/p2000/val'  # Caminho da pasta 'ROI'
destination_dir = 'D:/CloudSEN12+/p2000/Imagens-Brasil'  # Caminho do diretório de destino

# Lista de trechos que devem estar no nome da imagem
substrings_Am = ['T18M', 'T19M', 'T20M', 'T21M', 'T22M','T19N', 'T20N', 'T21N', 'T22N']
substrings_Br = ['T18M', 'T19M', 'T20M', 'T21M', 'T22M', 'T23M', 'T24M',
                'T19N', 'T20N', 'T21N', 'T22N',
                'T20L', 'T21L', 'T22L', 'T23L', 'T24L',
                'T21K', 'T22K', 'T23K', 'T24K',
                'T21J', 'T22J']  # Adicione os trechos específicos aqui

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Percorre todas as subpastas e arquivos dentro da pasta 'ROI'
for root, dirs, files in os.walk(source_dir):
    for file in files:
        # Verifica se o arquivo é uma imagem e contém os trechos no nome
        if file.endswith(('.tif')) and any(sub in file for sub in substrings_Br):
            # Caminho completo do arquivo
            file_path = os.path.join(root, file)
            
            # Copia o arquivo para o diretório de destino
            shutil.copy(file_path, destination_dir)

print("Imagens específicas copiadas com sucesso!")


Imagens específicas copiadas com sucesso!


In [5]:
import os
import shutil

def move_files_to_root(root_dir):
    # Percorre o diretório raiz (train, val, test)
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            root_file_path = os.path.join(root_dir, file)

            if subdir != root_dir:  # Não mover se já estiver na raiz
                if not os.path.exists(root_file_path):  # Verifica se já existe o arquivo na raiz
                    shutil.move(file_path, root_dir)  # Mover arquivo
                else:
                    print(f"Arquivo {file} já existe na raiz. Ignorando duplicata.")

# Caminhos para train, val e test
train_dir = 'D:/dataset/images/train'
val_dir = 'D:/dataset/images/val'
test_dir = 'D:/dataset/images/test'

# Mover arquivos para a raiz das pastas
move_files_to_root(train_dir)
move_files_to_root(val_dir)
move_files_to_root(test_dir)

Arquivo 20191010T085901_20191010T091932_T32KRF.tif já existe na raiz. Ignorando duplicata.
Arquivo 20200510T151659_20200510T152614_T18LUM.tif já existe na raiz. Ignorando duplicata.
Arquivo 20200709T151709_20200709T151705_T18LUM.tif já existe na raiz. Ignorando duplicata.
Arquivo 20200512T150731_20200512T150841_T18LYL.tif já existe na raiz. Ignorando duplicata.
Arquivo 20200517T150719_20200517T150734_T18LYL.tif já existe na raiz. Ignorando duplicata.
Arquivo 20200531T042709_20200531T042958_T46RDR.tif já existe na raiz. Ignorando duplicata.
Arquivo 20191222T151659_20191222T151941_T18MXA.tif já existe na raiz. Ignorando duplicata.
Arquivo 20190108T142039_20190108T143433_T18FXK.tif já existe na raiz. Ignorando duplicata.
Arquivo 20191117T151711_20191117T151705_T18MXA.tif já existe na raiz. Ignorando duplicata.
Arquivo 20190629T184921_20190629T185707_T10SEH.tif já existe na raiz. Ignorando duplicata.
Arquivo 20200325T185011_20200325T185014_T10SEH.tif já existe na raiz. Ignorando duplicata.

In [8]:
import rasterio
import os
import numpy as np
from scipy.ndimage import label

def generate_bboxes(mask, class_id):
    # Identificar componentes conectados para a classe atual
    labeled_array, num_features = label(mask == class_id)
    boxes = []

    for region_id in range(1, num_features + 1):
        # Obter coordenadas das regiões conectadas
        indices = np.where(labeled_array == region_id)
        y_min, y_max = np.min(indices[0]), np.max(indices[0])
        x_min, x_max = np.min(indices[1]), np.max(indices[1])

        # Calcular centro, largura e altura (normalizados)
        x_center = (x_min + x_max) / 2 / mask.shape[1]
        y_center = (y_min + y_max) / 2 / mask.shape[0]
        width = (x_max - x_min) / mask.shape[1]
        height = (y_max - y_min) / mask.shape[0]

        boxes.append([class_id, x_center, y_center, width, height])
    
    return boxes

def create_yolo_labels(image_id, mask, output_dir):
    labels = []
    for class_id in range(4):  # Classes: Limpo, Nuvem, Nuvem fina, Sombra de nuvem
        labels.extend(generate_bboxes(mask, class_id))

    # Caminho para salvar o arquivo de labels
    label_path = os.path.join(output_dir, f'{image_id}.txt')
    os.makedirs(os.path.dirname(label_path), exist_ok=True)

    with open(label_path, 'w') as f:
        for label in labels:
            f.write(' '.join(map(str, label)) + '\n')

def save_image_with_13_bands(image_id, bands, output_image_dir):
    # Caminho para salvar a imagem com 13 bandas
    output_image_path = os.path.join(output_image_dir, f'{image_id}.tif')
    
    # Salvar a imagem com as 13 bandas
    with rasterio.open(
        output_image_path, 'w',
        driver='GTiff',
        height=bands.shape[1],
        width=bands.shape[2],
        count=bands.shape[0],
        dtype=bands.dtype) as dst:
        
        for i in range(1, 14):  # Salvar cada banda (1 a 13)
            dst.write(bands[i - 1], i)

def process_images(image_dir, output_image_dir, output_label_dir):
    for filename in os.listdir(image_dir):
        if filename.endswith('.tif'):
            image_path = os.path.join(image_dir, filename)
            
            # Abrir a imagem .tif
            with rasterio.open(image_path) as src:
                # Ler as 13 bandas e a máscara (14ª banda)
                bands = src.read([i for i in range(1, 14)])  # 13 bandas
                mask = src.read(14)  # Máscara

                # Gerar o ID da imagem (sem extensão)
                image_id = os.path.splitext(filename)[0]

                # Salvar a imagem com as 13 bandas
                save_image_with_13_bands(image_id, bands, output_image_dir)

                # Criar arquivos de labels no formato YOLO
                create_yolo_labels(image_id, mask, output_label_dir)


# Caminhos das imagens e labels
train_images = 'D:/dataset/images/train'
val_images = 'D:/dataset/images/val'
test_images = 'D:/dataset/images/test'

train_labels = 'D:/dataset/labels/train'
val_labels = 'D:/dataset/labels/val'
test_labels = 'D:/dataset/labels/test'

train_13bands = 'D:/dataset/13bands_images/train'
val_13bands = 'D:/dataset/13bands_images/val'
test_13bands = 'D:/dataset/13bands_images/test'

# Processar as imagens e gerar máscaras no formato YOLO e salvar as imagens com 13 bandas
process_images(train_images, train_13bands, train_labels)
process_images(val_images, val_13bands, val_labels)
process_images(test_images, test_13bands, test_labels)


In [3]:
import os
import rasterio
import numpy as np

def normalize_band(band):
    """Normaliza os valores de uma banda para o intervalo 0-255."""
    band_min = np.min(band)
    band_max = np.max(band)
    normalized_band = 255 * (band - band_min) / (band_max - band_min)
    return normalized_band.astype(np.uint8)

def save_rgb_image_as_tif(tif_file, output_file, bands):
    """Extrai bandas RGB, normaliza e salva a imagem RGB no formato TIFF."""
    with rasterio.open(tif_file) as src:
        # Lê as bandas especificadas
        rgb_bands = [src.read(band) for band in bands]
        
        # Normaliza cada banda para o intervalo 0-255
        rgb_bands_normalized = [normalize_band(band) for band in rgb_bands]
        
        # Empilha as bandas para formar uma imagem RGB
        rgb_image = np.stack(rgb_bands_normalized, axis=-1)
        
        # Configurações para salvar a imagem TIFF
        kwargs = src.meta.copy()
        kwargs.update({
            'count': 3,  # Número de bandas na imagem RGB
            'dtype': 'uint8',  # Tipo de dados das bandas
            'driver': 'GTiff'  # Formato do arquivo
        })

        # Salva a imagem como TIFF
        with rasterio.open(output_file, 'w', **kwargs) as dst:
            dst.write(rgb_image[:, :, 0], 1)  # Banda vermelha
            dst.write(rgb_image[:, :, 1], 2)  # Banda verde
            dst.write(rgb_image[:, :, 2], 3)  # Banda azul

def process_images_in_folder(input_folder, output_folder, bands):
    """Processa todas as imagens .tif em uma pasta e salva imagens RGB em TIFF."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for subdir, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.tif'):
                input_file = os.path.join(subdir, file)
                output_file = os.path.join(output_folder, file.replace('.tif', '_rgb.tif'))
                save_rgb_image_as_tif(input_file, output_file, bands)

# Bandas para RGB (exemplo: 2 para azul, 3 para verde, 4 para vermelho)
bands_rgb = [4, 3, 2]  # Bandas do Sentinel-2 para RGB

# Diretórios
directories = {
    'train': 'D:/dataset/images/train',
    'val': 'D:/dataset/images/val',
    'test': 'D:/dataset/images/test'
}

for key, input_folder in directories.items():
    output_folder = f'path/to/{key}'
    process_images_in_folder(input_folder, output_folder, bands_rgb)
    print(f"Processamento concluído para o diretório {key}.")


Processamento concluído para o diretório train.
Processamento concluído para o diretório val.
Processamento concluído para o diretório test.
